In [1]:
import os
import sys

sys.path.append('../src-py/')
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [ ]:
import csv
import random
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import torch
import transformers
from rouge_score import rouge_scorer
from transformers import BartTokenizer, BartForConditionalGeneration
from torch.nn import CrossEntropyLoss, MSELoss
import os
from model import MultiTaskBart
from model import OurModel
from utils import parse_df
import time
import sys
import tqdm
import pandas as pd
from datasets import load_dataset, load_metric, Dataset

import nltk
pd.set_option('display.max_colwidth', None)

In [ ]:
import prompted_conclusion_utils as conc_utils
from prompted_conclusion_utils import *

from transformers.generation_logits_process import * 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using GPU? ", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0))


data_dir = '../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/'
teacher_model_path='../data/output/stance_classification/best_model/'

batch_size=4

In [ ]:
#Teacher model
stance_classifier_teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_path)
stance_classifier_teacher_model     = AutoModelForSequenceClassification.from_pretrained(teacher_model_path)
arg_stance_pipeline = TextClassificationPipeline(model=stance_classifier_teacher_model, tokenizer=stance_classifier_teacher_tokenizer, framework='pt', task='stance_classification', device=0)

In [ ]:
#Our model versions with the best performing checkpoint on the validation set..
model_without_stance    = OurModel.load('../data/output/ca-final-models/mt-v4.baseline_1/trained_models/models-global-step-5000', 'facebook/bart-large',  model_config=transformers.AutoConfig.from_pretrained('facebook/bart-large'))
model_with_stance = OurModel.load('../data/output/ca-final-models/mt-v4.baseline_2/trained_models/models-global-step-4000', 'facebook/bart-large',  model_config=transformers.AutoConfig.from_pretrained('facebook/bart-large'))

In [25]:
tokenizer = transformers.AutoTokenizer.from_pretrained('facebook/bart-large')

model_without_stance.to(device)
model_with_stance.to(device)

_ = model_with_stance.eval()
_ = model_without_stance.eval()

In [26]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def get_stance_scores(sents1, sents2):
    #compute stance score using our trained model
    text_inputs = [x[0] + ' </s> ' + x[1] for x in zip(sents1, sents2)]
    stance_results = arg_stance_pipeline(text_inputs, truncation=True)
    stance_labels = [int(x['label'].split('_')[-1]) for x in stance_results]
    stance_scores = [x['score'] for x in stance_results]
    return sum(stance_labels)/len(stance_labels), stance_labels, stance_scores  #The score is the percentage of cases we generated a counter

def counters_coherence(post_conclusions, post_counters):
    post_counters = [nltk.sent_tokenize(x) for x in post_counters]
    conclusion_counter_sent_pairs = [(x[1], s) for x in zip(post_counters, post_conclusions) for s in x[0]]
    #print(conclusion_counter_sent_pairs)
    conclusions, counter_sents = zip(*conclusion_counter_sent_pairs)
    _, stance_labels, stance_scores = get_stance_scores(conclusions, counter_sents)
    stance_scores = [x[0] * -1 if x[1] == 0 else x[0] for x in zip(stance_scores, stance_labels)]

    #collect counter_scores
    counter_scores = []
    idx = 0
    #print(len(stance_scores))
    for i, post_counter in enumerate(post_counters):
        #print(len(post_counter))
        counter_scores.append(stance_scores[idx: idx + len(post_counter)])
        idx+=len(post_counter)
    
    #print(counter_scores)
    return [np.mean(s) for s in counter_scores]

def get_best_counters(conclusions, counters):
    #choose best counter
    best_counters = []
    for post_conclusions, post_counters  in zip(conclusions, counters):
        scores = counters_coherence(post_conclusions, post_counters)
        best_counters.append((post_conclusions[np.argmax(scores)], post_counters[np.argmax(scores)]))
        
    return best_counters

def generate_multiple_counters(model, tokenizer, df, conclusion_gen_kwargs, argument_gen_kwargs, 
                               output_clm='post_counters', post_concepts_clm='post_concepts', num_candidates=6, batch_size=8):
    
    post_counters = []
    for i in range(num_candidates):
        control_codes = [x[i] if i < len(x) else '' for x in df[post_concepts_clm].tolist()]
        premises = df['post'].tolist()
        
        print(control_codes[:3])
        conclusions, counters   = generate_two_seq_prompted_counters(model, tokenizer, premises, 
                                                                     control_codes, conclusion_gen_kwargs, argument_gen_kwargs, 
                                                                     batch_size=batch_size)
                
        post_counters.append(zip(control_codes, conclusions, counters))

    df[output_clm] = list(map(list, zip(*post_counters))) # transpose

    return df


def generate_counters(model, tokenizer, data_loader, argument_gen_kwargs, conclusion_gen_kwargs, skip_special_tokens=True):
    processors = LogitsProcessorList()
    generated_counter_arguments = []
    generated_conclusions = []
    model.eval()
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            generated_argument_tokens   = model.generate_counter_argument(input_ids, attention_mask, argument_gen_kwargs, processors)
            generated_conclusion_tokens = model.generate_conclusion(input_ids, attention_mask, conclusion_gen_kwargs, processors)
                        
            generated_argument_tokens = generated_argument_tokens.cpu().numpy()
            decoded_arguments = tokenizer.batch_decode(generated_argument_tokens, skip_special_tokens=skip_special_tokens)
            
            generated_conclusion_tokens = generated_conclusion_tokens.cpu().numpy()
            decoded_conclusions = tokenizer.batch_decode(generated_conclusion_tokens, skip_special_tokens=skip_special_tokens)
            
            generated_counter_arguments +=decoded_arguments
            generated_conclusions +=decoded_conclusions
            
    return generated_conclusions, generated_counter_arguments

In [27]:
conclusion_gen_kwargs = {
    "do_sample": True, 
    "max_length":50,
    "top_p":0.95, 
    "num_beams":1,
}

argument_gen_kwargs = {
    "do_sample": True, 
    "max_length":100,
    "min_length":50,
    "top_p":0.95, 
    "no_repeat_ngram_size":3,
    "top_k": 50,
    "num_beams":1,
}

In [28]:
#df = pd.read_pickle(data_dir + 'test_conclusion_all_preprocessed.pkl')[['post_id', 'title', 'post', 'counter']]
#df = pd.read_pickle('../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed_multi_conclusions.pkl')[['post_id', 'title', 'post_concepts2', 'post', 'counter']]
df = pd.read_pickle('../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_all_preprocessed_multi_conclusions.pkl')[['post_id', 'title', 'post_concepts2', 'post', 'counter']]
df['post'] = df.post.apply(lambda x: ' '.join(x))

In [36]:
df.head()

post_id  \
410850  t3_a20l8s   
410858  t3_a22bn2   
410902  t3_a22nv4   
410910  t3_a22xd0   
410916  t3_a22y6n   

                                                                                                   title  \
410850                                                              people should come with instructions   
410858   People should not be heavily criticized for things they put on social media in the distant past   
410902                                                      We shouldn't focus on slowing climate change   
410910               The Australian PM was right to tell students to stop activism around global warming   
410916                                            Feeding cats or dogs a diet with meat is indefensible.   

                                                                                                    post_concepts2  \
410850         [Algorithm, Extraversion and introversion, People, Secrecy, Recommender system, Existential crisis]   
410858               [Internet, Black and white, Adolescence, Time, Reason, Belief, People, Person, Opinion, Jame]   
410902                                  [Climate change, Economics, Health, Environmental issues, People, Cutting]   
410910  [Climate change, Science, Activism, Government, Economist, Poverty, Scientist, Planet, Public policy, Fat]   
410916                                                           [Animal, Dog, Cat, Veganism, People, Animal feed]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [ ]:
df = generate_multiple_counters(model_without_stance, 
                                                   tokenizer, df, conclusion_gen_kwargs, argument_gen_kwargs, 
                                                   output_clm='post_counters', 
                                                   post_concepts_clm='post_concepts2', 
                                                   num_candidates=10, 
                                                   batch_size=16)

['Science', 'Law', 'Military']


Parameter 'function'=<function generate_two_seq_prompted_counters.<locals>.<lambda> at 0x7f8d4c1ce940> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

100%|██████████| 125/125 [05:02<00:00,  2.42s/it]

['Religion', 'Sovereignty', 'Transgender']


...

  0%|          | 0/2 [00:00<?, ?ba/s]

100%|██████████| 125/125 [04:53<00:00,  2.35s/it]

['', '', 'Secretary']


  0%|          | 0/2 [00:00<?, ?ba/s]

 40%|████      | 50/125 [01:57<02:53,  2.32s/it]

In [ ]:
df[['title', 'post_concepts2', 'post_counters']].head()

In [ ]:
conclusions = [[x[1] for x in row] for row in df.post_counters.tolist()]
counters = [[x[2] for x in row] for row in df.post_counters.tolist()]

In [ ]:
best_conclusions, best_counters = zip(*get_best_counters(conclusions, counters))

In [ ]:
df['pred_counter_arguments_no_stance'] = best_counters
df['pred_conclusions_no_stance'] = best_conclusions

In [ ]:
df[['title', 'post_counters', 'pred_conclusions_no_stance', 'pred_counter_arguments_no_stance']].head()

In [ ]:
df.to_pickle('../multitask-counter-arg-generation/data/output/ca-final-models/mt-v4/results/test_prompts_preds_df.pkl')

#### Predict single counters:

In [30]:
conclusion_gen_kwargs = {
    "do_sample": True, 
    "max_length":50,
    "top_p":0.95, 
    "num_beams":1,
}

argument_gen_kwargs = {
    "do_sample": True, 
    "max_length":100,
    "min_length":50,
    "top_p":0.95, 
    "no_repeat_ngram_size":3,
    "top_k": 50,
    "num_beams":4,
}

In [31]:
ds = Dataset.from_pandas(df[['post']])
ds = ds.map(lambda a: tokenizer(a['post'], padding='max_length', max_length=512, truncation=True), 
                                   remove_columns=[ '__index_level_0__'], batched=True)

ds.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(ds, batch_size=batch_size)

  0%|          | 0/9 [00:00<?, ?ba/s]

In [32]:
no_stance_conclusions, no_stance_counter_arguments = generate_counters(model_without_stance, tokenizer, dataloader, argument_gen_kwargs, conclusion_gen_kwargs)

In [33]:
df['single_pred_counter_arguments_no_stance'] = no_stance_counter_arguments
df['single_pred_conclusion_no_stance'] = no_stance_conclusions

In [35]:
df[['title', 'single_pred_counter_arguments_no_stance']].head()

,title,single_pred_counter_arguments_no_stance
410850,people should come with instructions,"i think you're missing out on a lot of the benefits of social interaction. first off, social interaction is not the only way to meet people, but it's the most efficient way to find them. if you're looking for a relationship, you're not going to be able to find one that suits you because you're going to have to be a little more selective about the people you meet. second, you'll be more likely to meet someone who doesn't like you"
410858,People should not be heavily criticized for things they put on social media in the distant past,"people change all the time. people change their views on a lot of things. their views change over time, and change as they get older. you can't expect someone to change in 10 years. you have to wait until the person you are today to change your view. if you don't change your views today, you will not be able to change them in the future."
410902,We shouldn't focus on slowing climate change,"climate change is real, but it's not a bad thing. the problem is that we don't know what to do about it. we can't predict what will happen in the future, and we have no way of knowing if it will happen at all. the best we can do is try to mitigate it as much as we can. we need to be prepared for it, but we can only do so much. if we want to fight climate change, we have to"
410910,The Australian PM was right to tell students to stop activism around global warming,"i think you're missing a few things:1. climate change is not something that will happen in the near future. 2. we have no way of knowing if it will happen or not. if it does, we can't do anything about it except wait and see. 3. it is not going to happen until the middle of the 20th century. there is no way to know if climate change will happen before then. 4. we are not in a position to"
410916,Feeding cats or dogs a diet with meat is indefensible.,"i'm a vegan, but i don't think veganism is the answer. veganism doesn't make sense to me. it's a lifestyle choice, not a moral choice. i'm not sure why you're trying to make a distinction between veganism and veganism."


In [26]:
df.to_pickle('../multitask-counter-arg-generation/data/output/ca-final-models/mt-v4/results/test_prompts_preds_df.pkl')

----------

In [10]:
df = pd.read_pickle('../multitask-counter-arg-generation/data/output/ca-final-models/mt-v4/results/test_prompts_preds_df.pkl')